In [ ]:
###CONFIGURANDO TABELAS###

import pandas as pd
import math
from datetime import datetime, timedelta

# Caminho para salvar o arquivo no Google Drive
file_path = r'C:\Users\henri\OneDrive\Desktop\DIVERSOS\OLIST/'

In [ ]:
###CARREGANDO ARQUIVOS###

datas = pd.read_csv(file_path+'silver/dim_data.csv')
produtos = pd.read_csv(file_path+'silver/produtos.csv')
entregas = pd.read_csv(file_path+'silver/dim_entregas.csv')
pedidos = pd.read_csv(file_path+'silver/fato_pedidos.csv')
transacoes = pd.read_csv(file_path+'silver/dim_transacao.csv')
localizacoes = pd.read_csv(file_path+'silver/dim_localizacoes.csv')

datas = datas[['dt_util'	,'dt_data',	'dt_diasemana',	'dt_diasemana_ord',	'dt_mesano',	'dt_mesano_ord',	'dt_trimestre',	'dt_trimestre_ord',	'dt_ano']]

produtos = produtos.drop('Unnamed: 0', axis = 1)
entregas = entregas.drop('Unnamed: 0', axis = 1)
pedidos = pedidos.drop('Unnamed: 0', axis = 1)
transacoes = transacoes.drop('Unnamed: 0', axis = 1)
localizacoes = localizacoes.drop('Unnamed: 0', axis = 1)

In [ ]:
###CRIAR TABELA ENTREGAS###

produtos.head()
produtos = produtos[['idproduto', 'categoria']]
carga = pd.merge(pedidos, produtos, on='idproduto', how='left')
carga = carga[['idpedido', 'categoria']]

entregas = pd.merge(carga, entregas, on='idpedido', how='inner')
entregas = entregas[['idpedido', 'categoria', 'cepcliente', 'cepvendedor']]

In [ ]:
###CRIAR TABELA FATO###

#processamento distancias

coordenadas = entregas[['idpedido', 'cepcliente', 'cepvendedor']]

entregas_pedidos_vendedor = pd.merge(coordenadas, localizacoes, left_on='cepvendedor', right_on='cep', how='left')
entregas_pedidos_vendedor.rename(columns={'latitude': 'latitude_vendedor', 'longitude': 'longitude_vendedor'}, inplace=True)
entregas_pedidos_completo = pd.merge(entregas_pedidos_vendedor, localizacoes, left_on='cepcliente', right_on='cep', how='left')
entregas_pedidos_completo.rename(columns={'latitude': 'latitude_cliente', 'longitude': 'longitude_cliente'}, inplace=True)

coordenadas = entregas_pedidos_completo[['idpedido', 'latitude_vendedor', 'longitude_vendedor', 'latitude_cliente', 'longitude_cliente']]
coordenadas.head()

##calcular coordenadas

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Raio da Terra em quilômetros

    # Converter graus para radianos
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Diferença das latitudes e longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Aplicar fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calcular a distância
    distance = R * c
    return distance


def calculate_distance(row):
    lat_vendedor = row['latitude_vendedor']
    lon_vendedor = row['longitude_vendedor']
    lat_cliente = row['latitude_cliente']
    lon_cliente = row['longitude_cliente']

    # Verificar se as coordenadas do cliente não são NaN
    if pd.notna(lat_cliente) and pd.notna(lon_cliente):
        distance = haversine(lat_vendedor, lon_vendedor, lat_cliente, lon_cliente)
        return distance
    else:
        return None

# Aplicar a função calculate_distance ao DataFrame usando apply
coordenadas['distancia_km'] = coordenadas.apply(calculate_distance, axis=1)

coluna = coordenadas[['idpedido', 'distancia_km']]

trans = pd.merge(pedidos, coluna, on='idpedido', how='left')

trans = trans[['idpedido', 'idvendedor', 'idcliente', 'preco', 'frete', 'volume', 'massa', 'distancia_km', 'ped_diasentrega', 'ped_diasprazo']]